#Sentiment & Thematic Analysis

In [11]:

import pandas as pd
from textblob import TextBlob

from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")


In [12]:

# Load cleaned review data
df = pd.read_csv('../data/cleaned_reviews.csv')
df.head()

,review,rating,date,bank,source
0,really am happy to this app it is Siple to use...,5,2025-06-07,Commercial Bank of Ethiopia,Google Play
1,I liked this app. But the User interface is ve...,2,2025-06-07,Commercial Bank of Ethiopia,Google Play
2,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,Commercial Bank of Ethiopia,Google Play
3,what is this app problem???,1,2025-06-05,Commercial Bank of Ethiopia,Google Play
4,the app is proactive and a good connections.,5,2025-06-05,Commercial Bank of Ethiopia,Google Play


In [13]:
def get_textblob_sentiment(text):
    polarity = TextBlob(str(text)).sentiment.polarity
    if polarity > 0:
        return 'Positive'
    elif polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'


df['sentiment'] = df['review'].apply(get_textblob_sentiment)

df['sentiment_score'] = df['review'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

print(df[['review', 'sentiment', 'sentiment_score']].head(10))
print(df['sentiment'].value_counts())
#print sample from each sentiment
print(df[df['sentiment'] == 'Negative'][['review', 'sentiment_score']].head())
print(df[df['sentiment'] == 'Positive'][['review', 'sentiment_score']].head())
print(df[df['sentiment'] == 'Neutral'][['review', 'sentiment_score']].head())

print(df['sentiment_score'].describe())


                                              review sentiment  \
0  really am happy to this app it is Siple to use...  Positive   
1  I liked this app. But the User interface is ve...  Positive   
2  "Why don’t your ATMs support account-to-accoun...  Negative   
3                        what is this app problem???   Neutral   
4       the app is proactive and a good connections.  Positive   
5    I cannot send to cbebirr app. through this app.   Neutral   
6                                               good  Positive   
7                                     not functional   Neutral   
8  everytime you uninstall the app you have to re...  Positive   
9  አካውንት የምናስገባበት ቦታ ስም ጽፈን ነው ከዚህ በፊት የላክንባቸውን አ...   Neutral   

   sentiment_score  
0         0.800000  
1         0.066667  
2        -0.125000  
3         0.000000  
4         0.700000  
5         0.000000  
6         0.700000  
7         0.000000  
8         0.234167  
9         0.000000  
sentiment
Positive    3585
Neutral     234

In [14]:
import torch
print(torch.__version__)
print(torch.cuda.is_available()) 

2.7.1+cpu
False


In [17]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def get_bert_sentiment(text):
    result = sentiment_pipeline(text[:512])[0]
    return result['label'], result['score']

bert_results = df['review'].apply(get_bert_sentiment)
df['bert_sentiment'], df['bert_score'] = zip(*bert_results)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/pytorch_model.bin: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


ValueError: Could not load model distilbert-base-uncased-finetuned-sst-2-english with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "c:\PY\bank-app-review-analysis\venv\Lib\site-packages\urllib3\response.py", line 754, in _error_catcher
    yield
  File "c:\PY\bank-app-review-analysis\venv\Lib\site-packages\urllib3\response.py", line 879, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "c:\PY\bank-app-review-analysis\venv\Lib\site-packages\urllib3\response.py", line 862, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ~~~~~~~~~~~~~^^^^^
  File "C:\Progr

In [ ]:

sentiment_summary = df.groupby(['bank', 'rating'])[['sentiment_score', 'bert_score']].mean().reset_index()
sentiment_summary.to_csv('../outputs/sentiment_summary.csv', index=False)


In [ ]:

def preprocess(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

df['cleaned_review'] = df['review'].apply(preprocess)


In [ ]:

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=100)
tfidf_matrix = tfidf.fit_transform(df['cleaned_review'])
keywords = tfidf.get_feature_names_out()
keywords[:20]


In [ ]:

themes = {
    "Transaction Issues": ["transaction", "failed", "deposit", "money", "transfer"],
    "Login/Access": ["login", "pin", "reset", "password"],
    "UI/UX": ["interface", "easy to use", "navigation", "layout"],
    "Performance": ["crash", "slow", "freeze", "lag"],
    "Feature Requests": ["add", "feature", "option", "support"]
}

def assign_theme(review):
    assigned = []
    for theme, keywords in themes.items():
        if any(keyword in review.lower() for keyword in keywords):
            assigned.append(theme)
    return assigned if assigned else ["Other"]

df['themes'] = df['cleaned_review'].apply(assign_theme)


In [ ]:

df[['review', 'sentiment', 'sentiment_score', 'bert_sentiment', 'bert_score', 'rating', 'bank', 'themes']]
  .to_csv('../outputs/sentiment_output.csv', index=False)
